In [48]:
import sys
sys.path.append('../../../../Scripts/')

In [49]:
import text_preprocessing_v2 as tp2
import cleaning_twitter_data as ctd
import baseline_model as base
import pandas as pd, numpy as np

## Data

In [50]:
data_training_path = '../Datasets/homomex_training.csv'

## Preprocessing

In [51]:
data = pd.read_csv(data_training_path)

In [55]:
dataframes = [data]
columns = ['tweets']
dictionary_list = [{'P': 0, 'NP': 1, 'NR': 2}]

In [56]:
clean = ctd.CleaningTwitterData(csv_path=data_training_path,
                                text_column='tweets', 
                                language='spanish', 
                                remove_stopwords=False, 
                                is_dataframe=True, 
                                emoji_path=None, 
                                dataframes=dataframes, 
                                columns=columns, 
                                dictionary_list=dictionary_list)

In [57]:
data = clean.clean_twitter_data(csv_path=data_training_path,
                                text_column='tweets', 
                                language='spanish')

In [59]:
data.label = data.label.fillna('NR')

In [61]:
data.label.replace(list(dictionary_list[0].keys()), 
            list(dictionary_list[0].values()), 
            inplace=True)

## Baseline

In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [76]:
seed = 42
model = LogisticRegression(random_state = seed, penalty = 'l2', 
                            solver = 'liblinear', max_iter = 1000)
X_train, X_val, y_train, y_val = train_test_split(data.tweets, data.label, 
                                                  test_size=0.2, 
                                                  random_state=seed)
target_names = ['LGBT+phobic (P)', 'not LGBT+phobic (NP)', 'not LGBT+related (NR)']

In [73]:
training_data = {'tweets': X_train.tolist(),
              'label': y_train.tolist()}
val_data = {'tweets': X_val.tolist(),
              'label': y_val.tolist()}

In [74]:
training_data = pd.DataFrame(training_data, columns=['tweets', 'label'])
val_data = pd.DataFrame(val_data, columns=['tweets', 'label'])

In [81]:
training_data.label.value_counts(), val_data.label.value_counts()

(1    3494
 2    1433
 0     673
 Name: label, dtype: int64,
 1    866
 2    345
 0    189
 Name: label, dtype: int64)

In [77]:
b = base.Baseline(train_data=training_data, 
                  test_data=val_data, 
                  x_label_column='tweets', 
                  y_label_column='label', 
                  target_names=target_names)

In [78]:
model, vectorizer = b.baseline(model=model, 
                               train_data=training_data, 
                               test_data=val_data, 
                               x_label_column='tweets', 
                               y_label_column='label', 
                               target_names=target_names)

Fitting label model
                       precision    recall  f1-score   support

      LGBT+phobic (P)       0.76      0.25      0.38       189
 not LGBT+phobic (NP)       0.79      0.95      0.86       866
not LGBT+related (NR)       0.87      0.73      0.79       345

             accuracy                           0.80      1400
            macro avg       0.81      0.65      0.68      1400
         weighted avg       0.80      0.80      0.78      1400

